In [ ]:
# default_exp utils.core

# Utils

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
%matplotlib inline

In [ ]:
#export
import numpy as np
import torch.nn as nn

from functools import partial
from collections import Iterable
from collections.abc import Generator
import re

import torch

In [ ]:
#export
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return o
    if isinstance(o, Iterable): return list(o)
    return [o]

In [ ]:
#export
def is_listy(x):
    "`isinstance(x, (tuple,list,L))`"
    return isinstance(x, (tuple, list, slice, Generator))

In [ ]:
#export
_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')


def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

In [ ]:
#export
def snakify_class_name(obj, cls_name):
    return camel2snake(re.sub(rf'{cls_name}$', '', obj.__class__.__name__) or cls_name.lower())

In [ ]:
#export
def get_default_device(use_cuda=None):
    "Return or set default device; `use_cuda`: None - CUDA if available; True - error if not availabe; False - CPU"
    b_GPU = use_cuda or (torch.cuda.is_available() and use_cuda is None)
    assert torch.cuda.is_available() or not b_GPU
    return torch.device(torch.cuda.current_device()) if b_GPU else torch.device('cpu')

In [ ]:
#export
def unsqueeze(x, dim=-1, n=1):
    "Same as `torch.unsqueeze` but can add `n` dims"
    for _ in range(n): x = x.unsqueeze(dim)
    return x

In [ ]:
#export
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

In [ ]:
#export
class NoneReduce():
    "A context manager to evaluate `loss_func` with none reduce."
    def __init__(self, loss_func): self.loss_func,self.old_red = loss_func,None

    def __enter__(self):
        if hasattr(self.loss_func, 'reduction'):
            self.old_red = self.loss_func.reduction
            self.loss_func.reduction = 'none'
            return self.loss_func
        else: return partial(self.loss_func, reduction='none')

    def __exit__(self, type, value, traceback):
        if self.old_red is not None: self.loss_func.reduction = self.old_red


In [ ]:
#export
def even_mults(start, stop, n):
    "Build log-stepped array from `start` to `stop` in `n` steps."
    if n==1: return stop
    mult = stop/start
    step = mult**(1/(n-1))
    return np.array([start*(step**i) for i in range(n)])

In [ ]:
#export
def generate_val_steps(val, n):
    if isinstance(val, slice):
        if val.start:
            val = even_mults(val.start, val.stop, n)
        else:
            val = [val.stop/10] * (n - 1) + [val.stop]
    vs = listify(val)
    if len(vs) == 1:
        vs = vs * n
    return vs

In [ ]:
#export
def format_time(t):
    "Format `t` (in seconds) to (h):mm:ss"
    t = int(t)
    h,m,s = t//3600, (t//60)%60, t%60
    if h!= 0: return f'{h}:{m:02d}:{s:02d}'
    else:     return f'{m:02d}:{s:02d}'

In [ ]:
from nbdev.export import *
notebook2script('utils.ipynb')

Converted utils.ipynb.
